In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa
import librosa.display
import IPython.display
import os
import cv2
from os import listdir

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, BatchNormalization
from keras import backend as K
from keras.models import load_model
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf


from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc 

import matplotlib
import gc
matplotlib.use('Agg') # No pictures displayed 
import pandas as pd

In [2]:
train_path = r"C:\Users\rudra\OneDrive\Desktop\animals\5_Class"
save_path = r"C:\Users\rudra\OneDrive\Desktop\animals\5_Class"
classes = listdir(train_path)

In [3]:
image_list, label_list = [] , []

In [4]:
try:
        print("[INFO] Loading images ...")
        image_folder_list = listdir(train_path)
        for image_folder in image_folder_list:
            print(f"[INFO] Processing {image_folder} ...")
            images_list = listdir(f"{train_path}/{image_folder}/")
            for samples in images_list[:]:
                image_directory = f"{train_path}/{image_folder}/{samples}"
                if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True or image_directory.endswith(".jpeg")==True:  
                    image_list.append(img_to_array(load_img(image_directory,target_size=(128,128))))
                    label_list.append(image_folder)
    
        print("[INFO] image extract completed")  
        print("total train samples: ")
        print(len(image_list))
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Aerial ...
[INFO] Processing Aquatic ...
[INFO] Processing Arboreal ...
[INFO] Processing Burrowing ...
[INFO] Processing Terrestrial ...
[INFO] image extract completed
total train samples: 
5399


In [5]:
# # Convert features (X) and labels (y) to Numpy arrays
X = np.array(image_list)/255
y = np.array(label_list)
image_list, label_list = [] , []

In [6]:
le = LabelEncoder()
Y = to_categorical(le.fit_transform(y))

In [7]:
n_classes = len(le.classes_)
print("Total number of classes: ", n_classes)

Total number of classes:  5


In [8]:
labels = classes
print(labels)

['Aerial', 'Aquatic', 'Arboreal', 'Burrowing', 'Terrestrial']


In [9]:
def create_model():
    num_rows = 128
    num_columns = 128
    num_channels = 3
    inputShape = (num_rows, num_columns, num_channels)
    chanDim = -1
    LR = 1e-3
    BATCH_SIZE = 32
    if K.image_data_format() == "channels_first":
        inputShape = (num_rows, num_columns, num_channels)
        chanDim = 1

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=inputShape))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(len(classes), activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=["accuracy"])
    
    return model

In [10]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix

def compute_confusion_matrix(y_true, y_pred,normalize=False):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return cm

In [11]:
def plot_confusion_matrix(cm,
                          classes, 
                          normalized=False, 
                          title=None, 
                          cmap=plt.cm.Blues,
                          size=(10,10)):
    fig, ax = plt.subplots(figsize=size)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalized else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    plt.show()

In [12]:
def acc_per_class(np_probs_array):    
    accs = []
    for idx in range(0, np_probs_array.shape[0]):
        correct = np_probs_array[idx][idx].astype(int)
        total = np_probs_array[idx].sum().astype(int)
        acc = (correct / total) * 100
        accs.append(acc)
    return accs

In [13]:
def visualize_conv_layers(model, img):
    layer_outputs = [layer.output for layer in model.layers if isinstance(layer, Conv2D)]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(np.expand_dims(img, axis=0))
    layer = 1
    for layer_activation in activations:
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // 16
        display_grid = np.zeros((size * n_cols, 16 * size))
        for col in range(n_cols):
            for row in range(16):
                channel_image = layer_activation[0, :, :, col * 16 + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(f'Conv Layer {layer}')
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        layer = layer + 1 

In [17]:
classification = {f"Fold":[], 
                  f"Accuracy": [], 
                  f"Precision": [],
                    f"Recall": [],
                    f"F-1 Score": []}
cv = KFold(n_splits = 3, shuffle=True, random_state=42)
fold_no = 1
acc_per_fold = []
loss_per_fold = []
EPOCHS =20
BATCH_SIZE = 32
gc.collect()
for train,test in cv.split(X,Y):
    model = create_model()
    print(f"[INFO] Training network fold {fold_no}...")
    history = model.fit(X[train],Y[train], batch_size=BATCH_SIZE,
                                  steps_per_epoch=len(X[train]) // BATCH_SIZE,
                                  epochs=EPOCHS, 
                                  verbose=1)
    gc.collect()
    print(f"[INFO] Saving Model fold {fold_no} ...")
    model.save(f"{save_path}/model_5_class_fold"+str(fold_no)+".h5")
    scores = model.evaluate(X[test],Y[test],verbose=0)
    acc_per_fold.append(scores[1]*100)
    loss_per_fold.append(scores[0])
    y_probs = model.predict(X[test])
    # Get predicted labels
    # yhat_probs = list(map(lambda x: 0 if (x<0.5) else 1,y_probs))
    yhat_probs = np.argmax(y_probs, axis=1)
    y_trues = y_trues = np.argmax(Y[test], axis=1)
    cm = confusion_matrix(y_trues, yhat_probs)
    plt.axis('off') # no axis
    # pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    plot_confusion_matrix(cm,
                              labels, 
                              normalized=False, 
                              title=f"Model Performance of 5_class fold: {fold_no}", 
                              cmap=plt.cm.Blues,
                              size=(12,12))
    plt.savefig(f'{save_path}/Model Performance of 5_class fold {fold_no}.jpg', bbox_inches=None, pad_inches=0)
    matplotlib.pyplot.close()
    gc.collect()
    # Accuracy 
    accuracy = accuracy_score(y_trues, yhat_probs) 
    # Precision 
    precision = precision_score(y_trues, yhat_probs,average=None) 
    # Recall 
    recall = recall_score(y_trues, yhat_probs,average=None) 
    # F1-Score 
    f1 = f1_score(y_trues, yhat_probs,average=None)
    classification["Fold"].append(f'fold - {fold_no}')
    classification["Accuracy"].append(accuracy)
    classification["Precision"].append(precision)
    classification["Recall"].append(recall)
    classification["F-1 Score"].append(f1)
    accuracies = acc_per_class(cm)
    pd.DataFrame({
        'CLASS': labels,
        'ACCURACY': accuracies
    }).sort_values(by="ACCURACY", ascending=False)
    re = classification_report(y_trues, yhat_probs, labels=[0,1,2,3,4], target_names=labels)
    print(re)
    fold_no = fold_no+1

[INFO] Training network fold 1...
Epoch 1/20
112/112 [==============================] - 7s 50ms/step - loss: 1.4624 - accuracy: 0.3954
Epoch 2/20
112/112 [==============================] - 5s 49ms/step - loss: 1.3660 - accuracy: 0.4424
Epoch 3/20
112/112 [==============================] - 5s 48ms/step - loss: 1.2643 - accuracy: 0.4900
Epoch 4/20
112/112 [==============================] - 5s 48ms/step - loss: 1.1883 - accuracy: 0.5200
Epoch 5/20
112/112 [==============================] - 5s 48ms/step - loss: 1.0458 - accuracy: 0.5997
Epoch 6/20
112/112 [==============================] - 5s 48ms/step - loss: 0.8485 - accuracy: 0.6801
Epoch 7/20
112/112 [==============================] - 5s 49ms/step - loss: 0.6369 - accuracy: 0.7710
Epoch 8/20
112/112 [==============================] - 5s 49ms/step - loss: 0.4753 - accuracy: 0.8315
Epoch 9/20
112/112 [==============================] - 5s 49ms/step - loss: 0.3083 - accuracy: 0.9008
Epoch 10/20
112/112 [==============================] - 5s

C:\Users\rudra\AppData\Local\Temp\ipykernel_2824\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


              precision    recall  f1-score   support

      Aerial       0.68      0.54      0.60       358
     Aquatic       0.75      0.65      0.70       334
    Arboreal       0.52      0.48      0.50       291
   Burrowing       0.39      0.54      0.45       254
 Terrestrial       0.64      0.68      0.66       563

    accuracy                           0.59      1800
   macro avg       0.59      0.58      0.58      1800
weighted avg       0.61      0.59      0.60      1800

[INFO] Training network fold 2...
Epoch 1/20
112/112 [==============================] - 7s 50ms/step - loss: 1.4945 - accuracy: 0.3730
Epoch 2/20
112/112 [==============================] - 6s 50ms/step - loss: 1.4017 - accuracy: 0.4177
Epoch 3/20
112/112 [==============================] - 6s 49ms/step - loss: 1.2627 - accuracy: 0.4912
Epoch 4/20
112/112 [==============================] - 6s 50ms/step - loss: 1.1306 - accuracy: 0.5618
Epoch 5/20
112/112 [==============================] - 6s 49ms/step - loss

C:\Users\rudra\AppData\Local\Temp\ipykernel_2824\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


              precision    recall  f1-score   support

      Aerial       0.53      0.57      0.55       315
     Aquatic       0.59      0.62      0.61       364
    Arboreal       0.55      0.42      0.48       303
   Burrowing       0.48      0.52      0.50       257
 Terrestrial       0.65      0.67      0.66       561

    accuracy                           0.58      1800
   macro avg       0.56      0.56      0.56      1800
weighted avg       0.58      0.58      0.58      1800

[INFO] Training network fold 3...
Epoch 1/20
112/112 [==============================] - 6s 49ms/step - loss: 1.4725 - accuracy: 0.3898
Epoch 2/20
112/112 [==============================] - 6s 49ms/step - loss: 1.2940 - accuracy: 0.4709
Epoch 3/20
112/112 [==============================] - 5s 49ms/step - loss: 1.2078 - accuracy: 0.5221
Epoch 4/20
112/112 [==============================] - 5s 49ms/step - loss: 1.1099 - accuracy: 0.5667
Epoch 5/20
112/112 [==============================] - 6s 49ms/step - loss

C:\Users\rudra\AppData\Local\Temp\ipykernel_2824\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


              precision    recall  f1-score   support

      Aerial       0.68      0.38      0.49       347
     Aquatic       0.54      0.60      0.57       322
    Arboreal       0.53      0.45      0.49       306
   Burrowing       0.47      0.59      0.52       268
 Terrestrial       0.60      0.71      0.65       556

    accuracy                           0.57      1799
   macro avg       0.57      0.55      0.54      1799
weighted avg       0.58      0.57      0.56      1799



In [18]:
df = pd.DataFrame(classification)
df.to_csv("Classification_report_of_5_class_classification.csv")

In [19]:
acc_per_fold

[59.50000286102295, 57.833331823349, 56.531405448913574]

In [20]:
loss_per_fold

[1.9017354249954224, 1.7930946350097656, 2.206712007522583]